In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import glob
from tqdm import tqdm
import threading
from keras.models import Sequential
from keras.layers import InputLayer, Dense, Conv2D, MaxPool2D, Flatten, Dropout, BatchNormalization, ReLU
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras.optimizers import Adam
import keras.backend

%matplotlib inline

In [12]:
data_dir = 'data/miniimagenet/'
train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'train')
valid_dir = os.path.join(data_dir, 'train')

In [23]:
IMG_SIZE = 64
IMG_CHANNELS = 3
N_CLASSES = len(os.listdir(train_dir))
N_INNER_CLASSES = 5
INNER_EPOCHS = 1
INNER_BATCH_SIZE = 10
N_INNER_STEPS = 8
INNER_LR = 0.001
OUTER_EPOCHS = 100000
OUTER_LR = 0.1
OUTER_BATCH_SIZE = 5
DISPLAY_FREQ = 50
N_VALID_TESTS = 50
MEM_LIMIT = 12 * 1000 * 1000 * 1000
IMG_BYTES = 98304
LOAD_MODEL = True
MODEL_NAME = 'Model-V3.hdf5'

## Define Image Loading Functions

In [14]:
img_cache = {}

In [15]:
def preprocess_img(img, size=IMG_SIZE):
    return cv2.resize(img, (size, size))

def load_img(class_dir=None, img_name=None, img_path=None):
    if img_path:
        img_name = img_path[img_path.rfind('/')+1:]
    else:
        img_path = os.path.join(class_dir, img_name[:img_name.find('_')], img_name)
    
    # If image is already loaded into cache return it
    if img_name in img_cache:
        return img_cache[img_name]
    else:
        img = cv2.imread(img_path)
        img = preprocess_img(img, size=IMG_SIZE)
        img = img / 255.
        if len(img_cache) * IMG_BYTES < MEM_LIMIT:
            img_cache[img_name] = img
        return img

## Building the DataFrame of Image Names

In [16]:
train_classes = np.asarray(os.listdir(train_dir))

cols = ['class', 'img_name']
df_train = pd.DataFrame(columns=cols)

for cat in train_classes:
    class_list = []
    cat_dir = os.listdir(os.path.join(train_dir, cat))
    for img_name in cat_dir:
        class_list.append([cat, img_name])
    tmp_df = pd.DataFrame(class_list, columns=cols)
    df_train = df_train.append(tmp_df)
    
df_train = df_train.sample(frac=1)
df_train.reset_index(inplace=True)
df_train.drop('index', inplace=True, axis=1)

## Same for Validation Set

In [17]:
valid_classes = np.asarray(os.listdir(train_dir))

cols = ['class', 'img_name']
df_valid = pd.DataFrame(columns=cols)

for cat in train_classes:
    class_list = []
    cat_dir = os.listdir(os.path.join(train_dir, cat))
    for img_name in cat_dir:
        class_list.append([cat, img_name])
    tmp_df = pd.DataFrame(class_list, columns=cols)
    df_valid = df_valid.append(tmp_df)
    
df_valid = df_train.sample(frac=1)
df_valid.reset_index(inplace=True)
df_valid.drop('index', inplace=True, axis=1)

## Functions for Generating Tasks and Formatting Data

In [18]:
def sample_task(data, n_classes=N_INNER_CLASSES):
    classes = np.random.choice(train_classes, size=n_classes, replace=False)
    task_indices = data['class'].map(lambda x: x in classes)
    task_data = data[task_indices]
    return task_data

def gen_batches(task_data, batch_size=INNER_BATCH_SIZE, data_dir=train_dir):
    task_data = pd.concat([task_data, pd.get_dummies(task_data['class'])], axis=1)
    while True:
        epoch_data = task_data.sample(frac=1)
        for i in range((len(epoch_data) // batch_size) - 1):
            X_names = epoch_data.iloc[i*batch_size:(i+1)*batch_size]['img_name'].values
            X = np.asarray([load_img(data_dir, img_name) for img_name in X_names])
            y = epoch_data.drop(['class', 'img_name'], axis=1).iloc[i*batch_size:(i+1)*batch_size].values
            
            yield X, y

## Building a Model

In [35]:
model = Sequential()
model.add(InputLayer(input_shape=(IMG_SIZE, IMG_SIZE, IMG_CHANNELS)))

for i in range(3):
    model.add(Conv2D(32, 3, padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPool2D((2, 2), padding='same'))
    model.add(ReLU())
    
model.add(Flatten())
model.add(Dense(N_INNER_CLASSES, activation='softmax'))

model.compile(optimizer=Adam(lr=INNER_LR),
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 64, 64, 32)        896       
_________________________________________________________________
batch_normalization_10 (Batc (None, 64, 64, 32)        128       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 32, 32, 32)        0         
_________________________________________________________________
re_lu_10 (ReLU)              (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 32, 32, 32)        9248      
_________________________________________________________________
batch_normalization_11 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 16, 16, 32)        0         
__________

In [36]:
if LOAD_MODEL:
    model.load_weights('models/' + MODEL_NAME)

## Training the Model

In [37]:
callbacks = [ReduceLROnPlateau(monitor='val_loss', factor=0.1, 
                               patience=15, verbose=0, 
                               mode='auto', min_delta=0.0001, 
                               cooldown=10, min_lr=0)]

In [40]:
hists = []

for i in range(OUTER_EPOCHS):
    
    initial_weights = model.get_weights()
    
    for _ in range(OUTER_BATCH_SIZE):
        task_train = sample_task(df_train)
        data_train = gen_batches(task_train, data_dir=train_dir)

        task_valid = sample_task(df_valid)
        data_valid = gen_batches(task_valid, batch_size=INNER_BATCH_SIZE*N_INNER_STEPS//5, data_dir=valid_dir)
        batch_valid = next(data_valid)
        
        hist = model.fit_generator(data_train,
                  steps_per_epoch=N_INNER_STEPS,
                  validation_data=batch_valid,
                  epochs=1,
                  callbacks=callbacks,
                  verbose=0)
        hists.append(hist.history)
        
    if i != 0 and i % DISPLAY_FREQ == 0:
        metrics = {'loss': 0, 'acc': 0, 'val_loss': 0, 'val_acc': 0, 'lr': 0}
        
        for hist in hists[-OUTER_BATCH_SIZE*DISPLAY_FREQ:]:
            for key in hist:
                metrics[key] += hist[key][0]
                
        print('Epoch', i)
        for key in metrics:
            metrics[key] /= OUTER_BATCH_SIZE*DISPLAY_FREQ
            print('{}: {:0.4f}'.format(key, metrics[key]))
        print()
        
    if i != 0 and i % (DISPLAY_FREQ*4) == 0:
        print('\nSaving Model\n')
        model.save('models/' + MODEL_NAME)
    
    model.set_weights([iw + OUTER_LR * (cw - iw)
                       for iw, cw in zip(initial_weights, model.get_weights())])

Epoch 50
loss: 1.3597
acc: 0.3935
val_loss: 1.3651
val_acc: 0.3960
lr: 0.0001

Epoch 100
loss: 1.3682
acc: 0.3970
val_loss: 1.3523
val_acc: 0.3940
lr: 0.0001

Epoch 150
loss: 1.3450
acc: 0.4021
val_loss: 1.3642
val_acc: 0.3978
lr: 0.0001

Epoch 200
loss: 1.3477
acc: 0.4068
val_loss: 1.3840
val_acc: 0.3715
lr: 0.0001


Saving Model

Epoch 250
loss: 1.3519
acc: 0.4025
val_loss: 1.3630
val_acc: 0.4055
lr: 0.0001

Epoch 300
loss: 1.3367
acc: 0.4089
val_loss: 1.3459
val_acc: 0.4070
lr: 0.0001

Epoch 350
loss: 1.3559
acc: 0.4024
val_loss: 1.3453
val_acc: 0.4090
lr: 0.0001

Epoch 400
loss: 1.3467
acc: 0.4066
val_loss: 1.3303
val_acc: 0.4263
lr: 0.0001


Saving Model

Epoch 450
loss: 1.3566
acc: 0.4004
val_loss: 1.3372
val_acc: 0.4135
lr: 0.0001

Epoch 500
loss: 1.3440
acc: 0.4023
val_loss: 1.3595
val_acc: 0.3925
lr: 0.0001

Epoch 550
loss: 1.3358
acc: 0.4085
val_loss: 1.3393
val_acc: 0.4128
lr: 0.0001

Epoch 600
loss: 1.3501
acc: 0.4058
val_loss: 1.3922
val_acc: 0.3880
lr: 0.0001


Saving Mod

Epoch 4900
loss: 1.3336
acc: 0.4092
val_loss: 1.3362
val_acc: 0.4045
lr: 0.0001

Epoch 4950
loss: 1.3411
acc: 0.3998
val_loss: 1.3025
val_acc: 0.4310
lr: 0.0001

Epoch 5000
loss: 1.3208
acc: 0.4126
val_loss: 1.3280
val_acc: 0.4183
lr: 0.0001


Saving Model

Epoch 5050
loss: 1.3011
acc: 0.4255
val_loss: 1.3664
val_acc: 0.3918
lr: 0.0001

Epoch 5100
loss: 1.3211
acc: 0.4185
val_loss: 1.3190
val_acc: 0.4230
lr: 0.0001

Epoch 5150
loss: 1.3279
acc: 0.4107
val_loss: 1.3277
val_acc: 0.4073
lr: 0.0001

Epoch 5200
loss: 1.3255
acc: 0.4138
val_loss: 1.3372
val_acc: 0.4095
lr: 0.0001


Saving Model

Epoch 5250
loss: 1.3184
acc: 0.4232
val_loss: 1.3267
val_acc: 0.4230
lr: 0.0001

Epoch 5300
loss: 1.3134
acc: 0.4190
val_loss: 1.3249
val_acc: 0.4215
lr: 0.0001

Epoch 5350
loss: 1.3362
acc: 0.4029
val_loss: 1.2992
val_acc: 0.4338
lr: 0.0001

Epoch 5400
loss: 1.3317
acc: 0.4086
val_loss: 1.3432
val_acc: 0.4123
lr: 0.0001


Saving Model

Epoch 5450
loss: 1.3314
acc: 0.4133
val_loss: 1.3392
val_acc: 0.

Epoch 9750
loss: 1.3268
acc: 0.4150
val_loss: 1.3435
val_acc: 0.3948
lr: 0.0001

Epoch 9800
loss: 1.3168
acc: 0.4161
val_loss: 1.3414
val_acc: 0.4143
lr: 0.0001


Saving Model

Epoch 9850
loss: 1.3031
acc: 0.4301
val_loss: 1.3288
val_acc: 0.4083
lr: 0.0001

Epoch 9900
loss: 1.3255
acc: 0.4212
val_loss: 1.3366
val_acc: 0.4178
lr: 0.0001

Epoch 9950
loss: 1.3451
acc: 0.4083
val_loss: 1.3170
val_acc: 0.4248
lr: 0.0001

Epoch 10000
loss: 1.3057
acc: 0.4233
val_loss: 1.3447
val_acc: 0.4003
lr: 0.0001


Saving Model

Epoch 10050
loss: 1.3206
acc: 0.4179
val_loss: 1.3314
val_acc: 0.4040
lr: 0.0001

Epoch 10100
loss: 1.3200
acc: 0.4137
val_loss: 1.3209
val_acc: 0.4255
lr: 0.0001

Epoch 10150
loss: 1.3390
acc: 0.4052
val_loss: 1.3466
val_acc: 0.4095
lr: 0.0001

Epoch 10200
loss: 1.3211
acc: 0.4209
val_loss: 1.3190
val_acc: 0.4283
lr: 0.0001


Saving Model

Epoch 10250
loss: 1.3222
acc: 0.4167
val_loss: 1.3648
val_acc: 0.3963
lr: 0.0001

Epoch 10300
loss: 1.3309
acc: 0.4149
val_loss: 1.3247
val_

KeyboardInterrupt: 

In [ ]:
model.save('models/' + MODEL_NAME)

In [ ]:
valid_score = 0
for i in tqdm(range(N_VALID_TESTS)):
    task = sample_task(df_valid)
    data = gen_batches(task)
    metrics = model.evaluate_generator(data, steps=N_INNER_STEPS, verbose=0)
    valid_score += metrics[1]
    
print('Accuracy:', valid_score / N_VALID_TESTS)

In [ ]:
a = hists[0]

In [ ]:
a.history

In [ ]:
scipy.signal.savgol_filter

In [ ]:
fig = plt.figure(figsize=(20, 8))

plt.plot(list(range(len(hists))), [hist['acc'][0] for hist in hists], lw=4)

In [ ]:
a = hists[0]

In [ ]:
a.history